# Abschluss

- Line bzw Barchart Master vs Bachelor u4ber die Jahre aggregiert für FB
- Treemap für alle Studiengänge

In [6]:
import plotly.io as pio

# Benutzerdefiniertes Template definieren
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            y=0.91,  # Titel weiter nach oben
            x=0.05,  # Links ausgerichtet
            xanchor="left",  # Linksbündig
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template

In [1]:
import pandas as pd

# Datei einlesen (ersetze 'data.xlsx' durch den tatsächlichen Dateinamen)
file_path = "data.xlsx"

# Alle Sheets einlesen, aber das erste Blatt ignorieren
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

# Erstes Blatt entfernen
sheets.pop(next(iter(sheets)))

# Container für die bereinigten DataFrames
cleaned_sheets = {}

# Durch alle Jahre iterieren und DataFrames bereinigen
for jahr, df in sheets.items():
    df.columns = df.columns.astype(str).str.strip()  # Spaltennamen bereinigen
    cleaned_sheets[jahr] = df  # Speichern der bereinigten Daten

In [3]:
abschluss_anteile = {}

for jahr, df in cleaned_sheets.items():
    # Studiengänge automatisch erkennen (Bachelor und Master)
    bachelor_studiengaenge = [col for col in df.columns if "Bachelor" in col]
    master_studiengaenge = [col for col in df.columns if "Master" in col]

    # Extrahiere die Gesamtanzahl der Studierenden pro Abschluss
    total_bachelor = df[df["Category"] == "Anzahl Studierende"][bachelor_studiengaenge].sum().sum()
    total_master = df[df["Category"] == "Anzahl Studierende"][master_studiengaenge].sum().sum()

    total_students = total_bachelor + total_master

    if total_students > 0:
        abschluss_anteile[jahr] = {
            "Bachelor": (total_bachelor / total_students) * 100,
            "Master": (total_master / total_students) * 100
        }
    else:
        abschluss_anteile[jahr] = {"Bachelor": 0, "Master": 0}

# DataFrame erstellen
abschluss_df = pd.DataFrame.from_dict(abschluss_anteile, orient="index").reset_index()
abschluss_df.rename(columns={"index": "Jahr"}, inplace=True)
abschluss_df = abschluss_df.sort_values(by="Jahr")

In [4]:
abschluss_df

,Jahr,Bachelor,Master
0,2013,63.706389,36.293611
1,2014,67.319983,32.680017
2,2015,69.597070,30.402930
3,2016,64.989138,35.010862
4,2017,59.922179,40.077821
5,2018,50.026525,49.973475
6,2019,58.325833,41.674167
7,2020,59.710316,40.289684
8,2021,57.366589,42.633411
9,2022,55.807028,44.192972


In [7]:
import plotly.express as px

# DataFrame für Plot umstrukturieren
abschluss_df_melted = abschluss_df.melt(id_vars="Jahr", var_name="Abschluss", value_name="Anteil (%)")

fig = px.line(
    abschluss_df_melted,
    x="Jahr",
    y="Anteil (%)",
    color="Abschluss",
    markers=True,
    title="Entwicklung der Abschlussarten (Bachelor vs. Master)",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

fig.update_traces(line=dict(width=3))  # Linie dicker machen

fig.update_layout(
    yaxis=dict(range=[0, 100]),  # Prozentwerte zwischen 0 und 100
    xaxis=dict(range=[2013, 2025])
)

fig.show()

In [34]:
import plotly.express as px


df_2024 = cleaned_sheets["2024"]

# Studiengänge automatisch erkennen (Bachelor und Master)
studiengaenge = [col for col in df_2024.columns if "Bachelor" in col or "Master" in col]

# Extrahiere die Anzahl der Studierenden pro Studiengang
studiengang_verteilung = df_2024[df_2024["Category"] == "Anzahl Studierende"][studiengaenge].T

studiengang_verteilung

,66
Bachelor Agrarwissenschaften,293
Bachelor Ernährungswissenschaften,493.0
Bachelor Nachwachsende Rohstoffe und Bioressourcen,69.0
Bachelor Ökotrophologie,429.0
Bachelor Umwelt und globaler Wandel,116.0
Master Agrar- und Ressourcenökonomie,53.0
Master Agrobiotechnologie,139.0
Master Ernährungswissenschaften,338.0
Master Getränketechnologie*,26.0
Master Informationstechnologie in den Agrar- und Umweltwissenschaften*,13.0


In [35]:
studiengang_verteilung.columns = ["Anzahl Studierende"]
studiengang_verteilung["Studiengang"] = studiengang_verteilung.index
studiengang_verteilung.index
studiengang_verteilung

,Anzahl Studierende,Studiengang
Bachelor Agrarwissenschaften,293,Bachelor Agrarwissenschaften
Bachelor Ernährungswissenschaften,493.0,Bachelor Ernährungswissenschaften
Bachelor Nachwachsende Rohstoffe und Bioressourcen,69.0,Bachelor Nachwachsende Rohstoffe und Bioressou...
Bachelor Ökotrophologie,429.0,Bachelor Ökotrophologie
Bachelor Umwelt und globaler Wandel,116.0,Bachelor Umwelt und globaler Wandel
Master Agrar- und Ressourcenökonomie,53.0,Master Agrar- und Ressourcenökonomie
Master Agrobiotechnologie,139.0,Master Agrobiotechnologie
Master Ernährungswissenschaften,338.0,Master Ernährungswissenschaften
Master Getränketechnologie*,26.0,Master Getränketechnologie*
Master Informationstechnologie in den Agrar- und Umweltwissenschaften*,13.0,Master Informationstechnologie in den Agrar- u...


In [37]:
# Treemap erstellen
fig = px.treemap(
    studiengang_verteilung,
    path=["Studiengang"],
    values="Anzahl Studierende",
    title="Studierendenverteilung nach Studiengang (2024)",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

fig.show()

In [63]:
import plotly.express as px


df_2024 = cleaned_sheets["2024"]

# Studiengänge automatisch erkennen (Bachelor und Master)
bachelor_studiengaenge = [col for col in df_2024.columns if "Bachelor" in col]
master_studiengaenge = [col for col in df_2024.columns if "Master" in col]

# Extrahiere die Anzahl der Studierenden pro Studiengang
studiengang_verteilung = df_2024[df_2024["Category"] == "Anzahl Studierende"][bachelor_studiengaenge + master_studiengaenge].T
studiengang_verteilung.columns = ["Anzahl Studierende"]
studiengang_verteilung["Studiengang"] = studiengang_verteilung.index

# Hinzufügen der übergeordneten Kategorie ("Bachelor" oder "Master")
studiengang_verteilung["Studienabschluss"] = studiengang_verteilung["Studiengang"].apply(
    lambda x: "Bachelor" if "Bachelor" in x else "Master"
)




# Farbkodierung für Studiengänge:  
# Bachelor-Studiengänge erhalten Farben aus "Set1", Master-Studiengänge aus "Set2"
bachelor_colors = px.colors.qualitative.Safe
master_colors = px.colors.qualitative.Safe

# Erstellen einer Farbkodierung für jeden Studiengang
color_map = {}
bachelor_count = 0
master_count = 0

for studiengang in studiengang_verteilung["Studiengang"]:
    if "Bachelor" in studiengang:
        color_map[studiengang] = bachelor_colors[bachelor_count % len(bachelor_colors)]
        bachelor_count += 1
    else:
        color_map[studiengang] = master_colors[master_count % len(master_colors)]
        master_count += 1

# Entferne "Bachelor" und "Master" aus den Studiengangsnamen
studiengang_verteilung["Studiengang"] = studiengang_verteilung["Studiengang"].str.replace("Bachelor ", "", regex=False)
studiengang_verteilung["Studiengang"] = studiengang_verteilung["Studiengang"].str.replace("Master ", "", regex=False)

# Treemap erstellen
fig = px.treemap(
    studiengang_verteilung,
    path=["Studienabschluss", "Studiengang"],  # Erstes Level: Bachelor/Master, Zweites Level: Studiengang
    values="Anzahl Studierende",
    title="Studierendenverteilung nach Studiengang und Abschluss (2024)",
    color="Studiengang",  # Jeder Studiengang bekommt eine eigene Farbe
    color_discrete_map=color_map,
    template="infoviz"
)

# Schriftgrößen anpassen
fig.update_layout(
    font=dict(size=18),  # Generelle Schriftgröße anpassen
)

# Schriftgröße für Labels innerhalb der Treemap anpassen
fig.update_traces(
    textinfo="label+value",  # Zeigt den Namen und die Anzahl an
    textfont=dict(size=18, color="black", weight="bold"),  # Größe der Texte innerhalb der Treemap
)
fig.update_layout(
    width=1200,  # Breite in Pixeln
    height=800   # Höhe in Pixeln
)

fig.show()

In [45]:
# Treemap erstellen
fig = px.bar(
    studiengang_verteilung.sort_values(by="Anzahl Studierende", ascending=False),
    x="Studiengang",
    y="Anzahl Studierende",
    title="Studierendenverteilung nach Studiengang (2024)",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

fig.show()

In [41]:
fig = px.treemap(
    studiengang_verteilung,
    path=["Studiengang"],
    values="Anzahl Studierende",
    title="Studierendenverteilung nach Studiengang (2024)",
    color="Studiengang",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

# Anpassungen für Form & Layout
fig.update_layout(
    shapes="remainder",
    margin=dict(t=50, l=25, r=25, b=25)
)

fig.update_traces(
    tiling=dict(packing="squarify"),
    marker=dict(pad=5, line=dict(width=2, color="white"))
)

fig.show()

ValueError: 
    Invalid value of type 'builtins.str' received for the 'shapes' property of layout
        Received value: 'remainder'

    The 'shapes' property is a tuple of instances of
    Shape that may be specified as:
      - A list or tuple of instances of plotly.graph_objs.layout.Shape
      - A list or tuple of dicts of string/value properties that
        will be passed to the Shape constructor

        Supported dict properties:
            
            editable
                Determines whether the shape could be activated
                for edit or not. Has no effect when the older
                editable shapes mode is enabled via
                `config.editable` or
                `config.edits.shapePosition`.
            fillcolor
                Sets the color filling the shape's interior.
                Only applies to closed shapes.
            fillrule
                Determines which regions of complex paths
                constitute the interior. For more i